In [1]:
import pandas as pd
import os
import numpy as np
import sys
sys.path.insert(0, '/home/xu.chao/cantera/build/python')
import cantera as ct
import shutil

In [2]:
benergy_c = np.arange(-7.5, -5.25, 0.25)
benergy_o = np.arange(-3.25, -5.5, -0.25)

In [3]:
parent_ps = []
total = 81
i = -1
paths = []
for c_binding_energy in benergy_c:
    i += 1
    j = -1
    index = total - ((7.5 - abs(c_binding_energy)) / 0.25 + 1) * 9
    for o_binding_energy in benergy_o:
        j += 1
        index += 1
        f_path = '{}_c{:.2f}o{:.2f}'.format(index, c_binding_energy, o_binding_energy)
        parent_ps.append(f_path)

In [10]:
# files = []
# for p in parent_ps:
#     shutil.rmtree(os.path.join(p, 'sim_data'))

In [6]:
files = []
for p in parent_ps:
    if os.path.exists(os.path.join(p, 'sim_data/rtol_1e-08_atol_1e-16_data.csv')):
        df = pd.read_csv(os.path.join(p, 'sim_data/rtol_1e-08_atol_1e-16_data.csv'))
        table_names = ['Ratio', 'SYNGAS Selec', 'SYNGAS Yield', 'CO Selectivity', 'CO % Yield', 'H2 Selectivity', 
                       'H2 % Yield', 'CH4 Conversion', 'H2O+CO2 Selectivity', 'H2O+CO2 yield', 'Exit Temp', 'Peak Temp',
                       'Dist to peak temp', 'O2 Conversion', 'Max CH4 Conv', 'Dist to 50 CH4 Conv']
        data_to_csv = []
        for i in range(len(df)):
            sens_data = calc(df.iloc[i,1:], type='sens')
            data_to_csv.append(sens_data)
        df_to_csv = pd.DataFrame(np.array(data_to_csv), columns=table_names)
        df_to_csv.to_csv(os.path.join(p, 'sim_data/therm_sens_ref.csv'))

In [7]:
# check if all the ref files are generated
files = []
for p in parent_ps:
    if not os.path.exists(os.path.join(p, 'sim_data/therm_sens_ref.csv')):
        print(p)

In [5]:
files = []
for p in parent_ps:
    if not os.path.exists(os.path.join(p, 'sim_data/rtol_1e-08_atol_1e-16_data.csv')):
        print(p)

75.0_c-7.50o-3.75


In [5]:
def calc(data, type='sens'):
    """
    Calculate properties of interest from the raw data
    :param data: the data
    :param type: 'sens' for sensitivity analyses
                 'output' for saving the output csv
                 'ratio' for plotting
    :return:
    """
    ratio, ch4_in, ch4_out, co_out, h2_out, h2o_out, co2_out, reference_exit_temp, reference_peak_temp, reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, reference_dist_to_50_ch4_conv = data

    ch4_depletion = ch4_in - ch4_out
    reference_ch4_conv = ch4_depletion / ch4_in 
    # negative sensitivity is higher selectivity
    reference_h2_sel = h2_out / (ch4_depletion * 2)  # Sensitivity definition 5: H2 selectivity
    if reference_h2_sel <= 0:
        reference_h2_sel = 1.0e-15  # selectivity can't be 0

    reference_co_sel = co_out / ch4_depletion  # Sensitivity definition 3: CO selectivity
    if reference_co_sel <= 0:
        reference_co_sel = 1.0e-15  # selectivity can't be 0

    reference_syngas_selectivity = reference_co_sel + reference_h2_sel  # Sensitivity definition 1: SYNGAS selectivity

    reference_syngas_yield = reference_syngas_selectivity * reference_ch4_conv  # Sensitivity definition 2: SYNGAS yield
    if reference_syngas_yield <= 0:
        reference_syngas_yield = 1.0e-15  # yield can't be 0

    reference_co_yield = co_out / ch4_in  # Sensitivity definition 4: CO % yield
    # reference_co_yield = reference_co_sel * reference_ch4_conv

    reference_h2_yield = h2_out / (2 * ch4_in)  # Sensitivity definition 6: H2 % yield
    # reference_h2_yield = reference_h2_sel * reference_ch4_conv

    # Sensitivity definition 8: H2O + CO2 selectivity
    reference_h2o_sel = h2o_out / (ch4_depletion * 2)
    reference_co2_sel = co2_out / ch4_depletion
    if reference_h2o_sel <= 0:
        reference_h2o_sel = 1.0e-15  # H2O selectivity can't be 0
    if reference_co2_sel <= 0:
        reference_co2_sel = 1.0e-15  # CO2 selectivity can't be 0
    reference_full_oxidation_selectivity = reference_h2o_sel + reference_co2_sel

    # Sensitivity definition 9: H2O + CO2 yield
    reference_full_oxidation_yield = reference_full_oxidation_selectivity * reference_ch4_conv
    
    sens_property = [ratio, reference_syngas_selectivity, reference_syngas_yield, reference_co_sel, reference_co_yield, reference_h2_sel, 
                     reference_h2_yield, reference_ch4_conv, reference_full_oxidation_selectivity, reference_full_oxidation_yield, 
                     reference_exit_temp, reference_peak_temp, reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, 
                     reference_dist_to_50_ch4_conv]
    
    ref_data = [ratio, ch4_in, ch4_out, co_out, h2_out, h2o_out, co2_out, reference_exit_temp, reference_peak_temp, 
                reference_peak_temp_dist, reference_o2_conv, reference_max_ch4_conv, reference_dist_to_50_ch4_conv]
    if type is 'sens':
        return sens_property
    elif type is 'ratio':
        return reference_co_sel, reference_h2_sel, reference_ch4_conv, reference_exit_temp, reference_o2_conv, reference_co2_sel, reference_h2o_sel
    elif type is 'gas_data':
        return ratio, reference
    else:
        return ref_data